<a href="https://colab.research.google.com/github/nabilBoudlal/BBGroup-Loyalty-Platform/blob/master/tesseract_con_craft_detection_e_preprocessing_immagini_di_testo_scritto_a_macchina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installazione dei pacchetti necessari:



In [ ]:
!pip install pytesseract==0.3.8
!apt install tesseract-ocr
!pip install craft-text-detector
!pip install torchvision==0.13

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2build2).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
import pytesseract
import shutil
import os
import random
import glob
import matplotlib.pyplot as plt
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image


def resize_image_to_dpi(image, target_dpi):
    # Ottieni la dimensione dell'immagine in pixel
    height, width, _ = image.shape

    # Calcola la dimensione fisica dell'immagine in pollici
    dpi = 96  # La risoluzione di default per le immagini caricate in OpenCV
    width_inch = width / dpi
    height_inch = height / dpi

    # Calcola il nuovo numero di pixel in base alla risoluzione desiderata in DPI
    target_width = int(target_dpi * width_inch)
    target_height = int(target_dpi * height_inch)

    # Ridimensiona l'immagine alla nuova dimensione
    resized_image = cv2.resize(image, (target_width, target_height), interpolation=cv2.INTER_CUBIC)

    # Ritorna l'immagine ridimensionata
    return resized_image


def preprocess_image(image_path, target_dpi):
    # Carica l'immagine utilizzando OpenCV
    image = cv2.imread(image_path)

    # Ridimensiona l'immagine al DPI desiderato
    resized_image = resize_image_to_dpi(image, target_dpi)

    # Converte l'immagine in scala di grigi
    gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

    kernel = np.ones((1, 1), np.uint8)
    img = cv2.dilate(gray_image, kernel, iterations=1)
    img = cv2.erode(gray_image, kernel, iterations=1)

    cv2.threshold(cv2.GaussianBlur(img, (5, 5), 0), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

    cv2.threshold(cv2.bilateralFilter(img, 5, 75, 75), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

    cv2.threshold(cv2.medianBlur(img, 3), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

    cv2.adaptiveThreshold(cv2.GaussianBlur(img, (5, 5), 0), 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)

    cv2.adaptiveThreshold(cv2.bilateralFilter(img, 9, 75, 75), 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)

    cv2.adaptiveThreshold(cv2.medianBlur(img, 3), 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)

    # Ritorna l'immagine preprocessata
    return img

 def show_crafted_image():
 image = '/content/outputs/image_text_detection.png'
 img= Image.open(image)
 img.show()


 def empty_folder():
 # Ottieni la lista dei file nella cartella
 folder_path= '/content/outputs/image_crops/'
 file_list = os.listdir(folder_path)

 # Itera sui file nella lista e elimina ciascun file
 for file_name in file_list:
    # Ottieni il percorso completo del file
    file_path = os.path.join(folder_path, file_name)

    # Verifica che il percorso sia un file (non una directory)
    if os.path.isfile(file_path):
        # Elimina il file
        os.remove(file_path)


def image_crafting(image_path, dpi):
  folder_path = '/content/outputs/image_crops/'

  # import craft functions
  from craft_text_detector import (
    read_image,
    load_craftnet_model,
    load_refinenet_model,
    get_prediction,
    export_detected_regions,
    export_extra_results,
    empty_cuda_cache
  )

  # set image path and export folder directory
  # can be filepath, PIL image or numpy array
  image= preprocess_image(image_path,dpi)
   #image= '/content/drive/MyDrive/Testo a stampa/B12_NEUBAUER-HUGO_20_0000.JPG'
  output_dir = 'outputs/'

  # read image
  image = read_image(image)

  # load models
  refine_net = load_refinenet_model(cuda=False)
  craft_net = load_craftnet_model(cuda=False)

# perform prediction
  prediction_result = get_prediction(
    image=image,
    craft_net=craft_net,
    refine_net=refine_net,
    text_threshold=0.7,
    link_threshold=0.4,
    low_text=0.4,
    cuda=False,
    long_size=1280
 )

 # export detected text regions
  exported_file_paths = export_detected_regions(
    image=image,
    regions=prediction_result["boxes"],
    output_dir=output_dir,
    rectify=True
 )

 # export heatmap, detection points, box visualization
  export_extra_results(
    image=image,
    regions=prediction_result["boxes"],
    heatmaps=prediction_result["heatmaps"],
    output_dir=output_dir
 )


def text_recognition():
 # Percorso della cartella contenente le immagini da elaborare
 folder_path = '/content/outputs/image_crops/'

 # Percorso del file di testo in cui verranno scritti i risultati
 output_file = 'risultati.txt'

 # Lista dei percorsi completi di tutte le immagini nella cartella
 image_paths = glob.glob(os.path.join(folder_path, '*.png'))
 image_paths = sorted(image_paths)

 # Apri il file di output in modalità scrittura
 with open(output_file, 'w') as file:
    # Itera su ogni immagine
    for image_path in image_paths:

        # Leggi l'immagine
        image = image_path
        img= Image.open(image)
        img.show()
        # Esegui l'OCR sull'immagine
        custom_config = r'--psm 7'  # Configurazione personalizzata per il rilevamento del testo
        text = pytesseract.image_to_string(image, config=custom_config)
        print(text)
        # Scrivi il risultato su file
        file.write(f"{text}")



Selezione dell'immagine tramire url e risoluzione desiderta (300 dpi valore consigliato da Tesseract)

In [ ]:
image_crafting('/content/drive/MyDrive/Testo a stampa/B12_NEUBAUER-HUGO_1_4_0001.JPG',300)

L'immagine segmentata e preprocessata viene passata all'Engine Tesseract che ne estrae il testo riconosciuto e lo salva in un file .txt

In [ ]:
text_recognition()

In [ ]:
empty_folder()